# Peek

First, let's see the structure of our files.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint as MC
from tensorflow.keras import backend as K
from sklearn.metrics import confusion_matrix, roc_curve, auc, recall_score, accuracy_score, balanced_accuracy_score, precision_score
import pickle
from IPython.display import FileLink



root = '/kaggle/input/rsna-str-pulmonary-embolism-detection'
for item in os.listdir(root):
    path = os.path.join(root, item)
    if os.path.isfile(path):
        print(path)

# Load Data

Next, we load all '.csv' files into memory and peek into their makeup.

In [ ]:
print('Reading train data...')
data = pd.read_csv("../input/rsna-str-pulmonary-embolism-detection/train.csv")
print(data.shape)
data.head()

In [ ]:
#Plot EDA
for col in data.columns[3:len(data.columns)]:
    proportion=[sum(data[col]), len(data)-sum(data[col])]
    proportion=pd.Series(proportion)
    #colors
    colors = ['#A9C4EB','#10739E']
    #explsion
    explode = (0.05,0.05)
    proportion.plot(kind='pie',colors = colors, labels=[True,False], 
        autopct='%1.1f%%', startangle=90, pctdistance=0.85, 
        explode = explode)
#     plt.pie(proportion, colors = colors, labels=[True,False], 
#             autopct='%1.1f%%', startangle=90, pctdistance=0.85, 
#             explode = explode)
    plt.title(col)
    #draw circle
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    # Equal aspect ratio ensures that pie is drawn as a circle
#     ax1.axis('equal')  
    plt.tight_layout()
    plt.show()

# Train/Test Split
Since this project is for academic purposes, we will split the give train set into train+test set, to evaluate performance of the model better.
We will split by StudyInstanceID which is the exam ID, so that images from the same exam don't show up in train and test.

In [ ]:
# Separate the labels from the test set and save for later
keys=['negative_exam_for_pe', 'qa_motion',
       'qa_contrast', 'flow_artifact', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1',
       'leftsided_pe', 'chronic_pe', 'true_filling_defect_not_pe',
       'rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate']

y=data['pe_present_on_image'].copy()
x=data.drop('pe_present_on_image',axis=1)

for key in keys:
    y=pd.concat([y,x[key]],axis=1)
    x=x.drop(key, axis=1)

In [ ]:
from sklearn.model_selection import GroupShuffleSplit,StratifiedShuffleSplit, GroupKFold

#train,test=train_test_split(data, test_size=0.2, random_state=42, shuffle=False)

#This split allows train and test sets to not have images from the same exam
splitter = GroupShuffleSplit(test_size=.20, random_state=42, n_splits=5)
# splitter = StratifiedShuffleSplit(test_size=.20, random_state=42, n_splits=2) #this split would shuffle images, both were tried but no better results
split = splitter.split(x,y, groups=data.StudyInstanceUID)
train_inds, test_inds = next(split)

train = data.iloc[train_inds]
test = data.iloc[test_inds]

In [ ]:
train.head()
# print('%PE positive images:', round(sum(train.pe_present_on_image)/len(train)*100),'%')

In [ ]:
test.head()
y_test=test['pe_present_on_image'].copy()
test=test.drop('pe_present_on_image',axis=1)

for key in keys:
    y_test=pd.concat([y_test,test[key]],axis=1)
    test=test.drop(key, axis=1)
    

print('%PE positive images:', round(sum(y_test.pe_present_on_image)/len(train)*100),'%\n',
      '# of PE positive images:', sum(y_test.pe_present_on_image))

In [ ]:
y_test.head()

After checking the keys, we will now work on the fuction to get the image array from a DICOM image. We will be using a code snippet by [eladwar](https://www.kaggle.com/eladwar) from his notebook [here](https://www.kaggle.com/eladwar/20-seconds-or-less).

In [ ]:
import vtk
from vtk.util import numpy_support
import cv2
import albumentations as albu

reader = vtk.vtkDICOMImageReader()

train_transform = [albu.RandomBrightnessContrast(p=0.3),
                   albu.VerticalFlip(p=0.5),
                   albu.HorizontalFlip(p=0.5),
                   albu.Downscale(p=1.0,scale_min=0.35,scale_max=0.75,),
                   albu.Resize(512, 512)]
transform=albu.Compose(train_transform)

def normalize_image(image):
    min = -1000
    max = 400
    image[image < min] = min
    image[image > max] = max
#     image = (image - min) / (max - min)
#     image = image.astype("float64")
    return image

def get_img(path):
    reader.SetFileName(path)
    reader.Update()
    _extent = reader.GetDataExtent()
    ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

    ConstPixelSpacing = reader.GetPixelSpacing()
    imageData = reader.GetOutput()
    pointData = imageData.GetPointData()
    arrayData = pointData.GetArray(0)
    ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
    ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')
    ArrayDicom = cv2.resize(ArrayDicom,(512,512))
    
    ArrayDicom = normalize_image(ArrayDicom)
    return ArrayDicom

After defining our image reader, we will test it with a sample DICOM image to load.

In [ ]:
#test read a dcom file and view it
fpath = "../input/rsna-str-pulmonary-embolism-detection/train/6897fa9de148/2bfbb7fd2e8b/be0b7524ffb4.dcm"
ds = get_img(fpath)

import matplotlib.pyplot as plt

#Convert dcom file to 8bit color
func = lambda x: int((2**15 + x)*(255/2**16))
int16_to_uint8 = np.vectorize(func)

def show_dicom_images(dcom,transform_img=False):
    f, ax = plt.subplots(1,2, figsize=(16,20))
    if transform_img:
        data_row_img = int16_to_uint8(dcom)
        data_row_img=transform(image=data_row_img)['image']
        title1='8-bit DICOM Transformed Image'
    else:
        data_row_img = int16_to_uint8(dcom)
        title1='8-bit DICOM Image'
        
    ax[0].imshow(data_row_img, cmap=plt.cm.bone)
    ax[1].imshow(ds, cmap=plt.cm.bone)
    #print(data_row_img)
    ax[0].axis('off')
    ax[0].set_title(title1)
    ax[1].axis('off')
    ax[1].set_title('16-bit DICOM Image')
    plt.show()


show_dicom_images(ds,False)

# Model Creation

I decided to try and use a pre-trained Xception model as a feature-extractor. To simplify my coding, I did a multi-output model with each output having one node activated by a sigmoid. Also, since we are dealing with numbers between 1 and 0, I decided to use binary_crossentropy as the loss function.

**Embedding Model Creation**

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Embedding, Dot, Reshape, Multiply

# inputs = Input((512, 512, 3))
# #x = Conv2D(3, (1, 1), activation='relu')(inputs)
# base_model = keras.applications.Xception(
#     include_top=False,
#     weights="imagenet"
# )

# base_model.trainable = False

# # First Branch of Neural Network - Classify presense of PE
# base_outputs = base_model(inputs, training=False)
# pool_outputs = keras.layers.GlobalAveragePooling2D()(base_outputs)
# drop_outputs = Dropout(0.25)(pool_outputs)
# outputs1_dense1 = Dense(1024, activation='relu')(drop_outputs)
# outputs1_dense2 = Dense(256, activation='relu')(outputs1_dense1)
# outputs1_dense3 = Dense(64, activation='relu')(outputs1_dense2)

# #First Output
# ppoi = Dense(1, activation='sigmoid', name='pe_present_on_image')(outputs1_dense3)
# indt = Dense(1, activation='sigmoid', name='indeterminate')(outputs1_dense3)
# # Second Branch of Neural Network - Classify severity and position of PE, based on the presence of PE

# #Embedding labels of pe prediction into the input of this model - the embedding weights will be learned during
# #backpropagation
# pe_input = Input((None, 1))
# embedding_size = 2048

# #input_dim is the batch_size which in this case is 1000*0.9 (0.10 is for validation)
# pe_embedding = Embedding(name='pe_embedding',input_dim = 900, output_dim = embedding_size, input_length=1)(ppoi)
# # multi=Multiply()([outputs1_dense2,pe_embedding])
# # merged=Dot(axes=1)([multi, multi])
# reshape=Reshape(target_shape = [embedding_size])(pe_embedding)
# merged=reshape*drop_outputs
# #Merging the embeddings
# #merged = Dot(name = 'dot_product', normalize = True, axes = 0)([drop_outputs,pe_embedding])
# outputs2_dense1 = Dense(256, activation='relu')(merged)
# outputs2_dense2 = Dense(128, activation='relu')(outputs2_dense1)
# outputs2_dense3 = Dense(64, activation='relu')(outputs2_dense2)
# #Outputs
# lspe = Dense(1, activation='sigmoid', name='leftsided_pe')(outputs2_dense3)
# rspe = Dense(1, activation='sigmoid', name='rightsided_pe')(outputs2_dense3)
# cnpe = Dense(1, activation='sigmoid', name='central_pe')(outputs2_dense3)
# cpe = Dense(1, activation='sigmoid', name='chronic_pe')(outputs2_dense3)
# aacpe = Dense(1, activation='sigmoid', name='acute_and_chronic_pe')(outputs2_dense3)

# model = Model(inputs=inputs, outputs={'pe_present_on_image':ppoi,
#                                       'leftsided_pe':lspe,
#                                       'chronic_pe':cpe,
#                                       'rightsided_pe':rspe,
#                                       'acute_and_chronic_pe':aacpe,
#                                       'central_pe':cnpe,
#                                       'indeterminate':indt})

# opt = keras.optimizers.Adam(lr=0.00001)


# model.compile(optimizer=opt,
#               loss='binary_crossentropy',
#               metrics=tf.keras.metrics.AUC())
# model.summary()
# model.save('pe_detection_model.h5')
# del model
# K.clear_session()
# gc.collect()

Create an embedding of the PE prediction (present/not present) and the image itself, that will be inputs to our second model, which will predict location and severity

**VGG Model Creation**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Embedding, Dot, Reshape, Multiply
inputs = Input((512, 512, 3))
vgg=keras.applications.VGG16(
    include_top=False,
    weights=None)

# First Branch of Neural Network - Classify presense of PE
vgg_outputs = vgg(inputs, training=True)
pool_outputs = keras.layers.GlobalAveragePooling2D()(vgg_outputs)
drop_outputs = Dropout(0.25)(pool_outputs)
outputs1_dense1 = Dense(1024, activation='relu')(drop_outputs)
outputs1_dense2 = Dense(256, activation='relu')(outputs1_dense1)
outputs1_dense3 = Dense(64, activation='relu')(outputs1_dense2)

#Outputs
ppoi = Dense(1, activation='sigmoid', name='pe_present_on_image')(outputs1_dense3)
indt = Dense(1, activation='sigmoid', name='indeterminate')(outputs1_dense3)
lspe = Dense(1, activation='sigmoid', name='leftsided_pe')(outputs1_dense3)
rspe = Dense(1, activation='sigmoid', name='rightsided_pe')(outputs1_dense3)
cnpe = Dense(1, activation='sigmoid', name='central_pe')(outputs1_dense3)
cpe = Dense(1, activation='sigmoid', name='chronic_pe')(outputs1_dense3)
aacpe = Dense(1, activation='sigmoid', name='acute_and_chronic_pe')(outputs1_dense3)

model = Model(inputs=inputs, outputs={'pe_present_on_image':ppoi,
                                      'leftsided_pe':lspe,
                                      'chronic_pe':cpe,
                                      'rightsided_pe':rspe,
                                      'acute_and_chronic_pe':aacpe,
                                      'central_pe':cnpe,
                                      'indeterminate':indt})

opt = keras.optimizers.Adam(lr=0.00001)


model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=tf.keras.metrics.AUC())
model.summary()
model.save('pe_detection_model.h5')
del model
K.clear_session()
gc.collect()

# Training Model

Before we can train our model, we would be needing an image generator. This is so that our training code would look much cleaner and nicer.

*custom_dcom_image_generator* is a function that constructs the batch on which we will train and test.
If train, it will yield (return a generator, iterable only once) with the X and Y, where X is a group of images
If test, it will yield the X on which we will predict.

By using yield, we only iterate through the batch once and don't save it in memory.

In [ ]:
def convert_to_rgb(array):
    array = array.reshape((512, 512, 1))
    return np.stack([array, array, array], axis=2).reshape((512, 512, 3))
    
def custom_dcom_image_generator(batch_size, dataset, test=False, debug=False):
    
    fnames = dataset[['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID']]
    
    if not test:
#         Y=dataset[classes]
        Y = dataset[['pe_present_on_image', 'leftsided_pe',
                     'chronic_pe', 'rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate'
                    ]]
        prefix = 'input/rsna-str-pulmonary-embolism-detection/train'
        
    else:
        prefix = 'input/rsna-str-pulmonary-embolism-detection/train'
    X = []
    batch = 0
    for st, sr, so in fnames.values:
        if debug:
            print(f"Current file: ../{prefix}/{st}/{sr}/{so}.dcm")

        dicom = get_img(f"../{prefix}/{st}/{sr}/{so}.dcm")
        
#         eightbit=int16_to_uint8(dicom)
#         transformed_img=transform(image=dicom)['image']
#         transformed_img=convert_to_rgb(transformed_img)
        
#         X.append(transformed_img)

        image = convert_to_rgb(dicom)
        X.append(image)
            
        del st, sr, so
        
        #If we reached the end of the batch
        if len(X) == batch_size:
            if test:
                #yield is used to save memory
                yield np.array(X)
                del X
            else:
                yield np.array(X), Y[batch*batch_size:(batch+1)*batch_size].values
                del X
                
            gc.collect()
            X = []
            batch += 1
        
    if test:
        yield np.array(X)
    else:
        yield np.array(X), Y[batch*batch_size:(batch+1)*batch_size].values
        del Y
    del X
    gc.collect()
    return

Next, we will be training our model with the train data. We will be using a small batch, fitted with 3 epochs (arbitrary choice) of training to minimize our RAM usage. This is due to our large model, which will eat up quite a large portion of our RAM.

We will also use sampling for our train data. This will shuffle the data.

In [ ]:
def train_model(model_path, train_data, train_size, batch_size, max_train_time, debug):
    #Train loop
    for n, (x, y) in enumerate(custom_dcom_image_generator(batch_size, train_data, False, debug)):
        

        if len(x) < 10: #Tries to filter out empty or short data
            break

        clear_output(wait=True)
        print("Training batch: %i - %i" %(batch_size*n, batch_size*(n+1)))
        model = load_model(model_path)
        hist = model.fit(
            x[:train_size], 
            #Y values are in a dict as there's more than one target for training output
            {'pe_present_on_image':y[:train_size, 0],
             'leftsided_pe':y[:train_size, 1],
             'chronic_pe':y[:train_size, 2],
             'rightsided_pe':y[:train_size, 3],
             'acute_and_chronic_pe':y[:train_size, 4],
             'central_pe':y[:train_size, 5],
             'indeterminate':y[:train_size, 6]},

            callbacks = checkpoint,

            validation_split=0.2,
            epochs=3,
            batch_size=8,
            verbose=debug
        )

        print("Metrics for batch validation:")
        model.evaluate(x[train_size:],
                       {'pe_present_on_image':y[train_size:, 0],
                        'leftsided_pe':y[train_size:, 1],
                        'chronic_pe':y[train_size:, 2],
                        'rightsided_pe':y[train_size:, 3],
                        'acute_and_chronic_pe':y[train_size:, 4],
                        'central_pe':y[train_size:, 5],
                        'indeterminate':y[train_size:, 6]
                       }
                      )

        try:
            for key in hist.history.keys():
                history[key] = np.concatenate([history[key], hist.history[key]], axis=0)
        except:
            for key in hist.history.keys():
                history[key] = hist.history[key]

        #To make sure that our model don't train overtime
        if time.time() - start >= max_train_time:
            print("Time's up!")
            break

        model.save('pe_detection_model.h5')
        del model, x, y, hist
        K.clear_session()
        gc.collect()
    
    
    return history, model

In [ ]:
history = {}
start = time.time()
train_data=train.sample(frac=1)
batch_size = 1000
debug = 0
#90% for training, 10% for validation
train_size = int(batch_size*0.9)

max_train_time = 3600 * 2 #hours to seconds of training

checkpoint = MC(filepath='../working/pe_detection_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
#Train loop
history,trained_model = train_model('../working/pe_detection_model.h5', train_data, train_size, batch_size, max_train_time, debug)
trained_model.save('pe_detection_model_trained.h5')


We will now look at the history of the training for our data. Since the data is trained across several different batches, there should be some spikes among the values reflected here.

In [ ]:
from IPython.display import FileLink
import csv, pickle

FileLink(r'pe_detection_model_trained.h5')

In [ ]:
with open('history.pkl', 'wb') as f:
    pickle.dump(history, f)
    
FileLink(r'history.pkl')

In [ ]:
#Import history file if needed
with open('../input/models/hist_file_embed_2048_lr0.00001_norm.pkl', 'rb') as f:
    history=pickle.load(f)

In [ ]:
for key in history.keys():
    if key.startswith('val'):
        continue
    else:
        epoch = range(len(history[key]))
        plt.plot(epoch, history[key]) #X=epoch, Y=value
        plt.plot(epoch, history['val_'+key])
        plt.title(key)
        if 'accuracy' in key:
            plt.axis([0, len(history[key]), -0.1, 1.1]) #Xmin, Xmax, Ymin, Ymax
        plt.legend(['train', 'validation'], loc='upper right')
        plt.show()

# End of Part 1

# Start of Part 2

# Prediction

We will now proceed to predict our data. We will use our own test set, obtained from the original train set of the submission, in order to obtain the metrics we want (ROC_AUC, F1)

In [ ]:
predictions = {}
stopper = 3600 * 4 #8 hours limit for prediction
pred_start_time = time.time()\

p, c = time.time(), time.time()
batch_size = 1000
    
l = 0
n = test.shape[0]

for x in custom_dcom_image_generator(batch_size, test, True, False):
    clear_output(wait=True)
    model = load_model("../input/models/embed_2048_lr0.00001_norm.h5")
#     model = trained_model
    preds = model.predict(x, batch_size=8, verbose=1)
    
    try:
        for key in preds.keys():
            predictions[key] += preds[key].flatten().tolist()
            
    except Exception as e:
        print(e)
        for key in preds.keys():
            predictions[key] = preds[key].flatten().tolist()
            
    l = (l+batch_size)%n
    print('Total predicted:', len(predictions['indeterminate']),'/', n)
    p, c = c, time.time()
    print("One batch time: %.2f seconds" %(c-p))
    print("ETA: %.2f" %((n-l)*(c-p)/batch_size))
    
    if c - pred_start_time >= stopper:
        print("Time's up!")
        break
    
    del model
    K.clear_session()
    
    del x, preds
    gc.collect()

We will now check the shape of each key in predictions to make sure we predicted everything.

In [ ]:
pred_file = "pred_file.pkl"
with open(pred_file,'wb') as f:
    pickle.dump(predictions, f)
FileLink(pred_file)

In [ ]:
predictions

### Import predictions if necessary

In [ ]:
import pickle 
with open('../input/predictions/pred_embed_2048_lr0.00001_norm.pkl','rb') as f:
    predictions=pickle.load(f)

In [ ]:
for key in predictions.keys():
    print(key, np.array(predictions[key]).shape)

Next, we will convert predictions into a dataframe based on the test dataframe. We will also copy all unique **StudyInstanceUID** for predicting later.

In [ ]:
for key in y_test.keys():
    print(key, np.array(y_test[key]).shape)

# Calculate the image-level test performance 

In [ ]:
def GetFinalScores(pred, y_test, agg=False):
    test_scores={}
    pred=round(pred)
    if not agg:
        keys=['pe_present_on_image', 'leftsided_pe','chronic_pe', 'rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate']
    else:
        keys=['negative_exam_for_pe', 'leftsided_pe','chronic_pe', 'rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate']
    for key in keys:
        fpr, tpr, thresholds = roc_curve(y_test[key], pred[key])
        roc_auc = auc(fpr, tpr)
        conf_matrix=confusion_matrix(y_test[key], pred[key])

#         precision = precision_score(y_test[key], pred[key])

        recall = recall_score(y_test[key], pred[key])

        accuracy = balanced_accuracy_score(y_test[key], pred[key])
        
        test_scores[key]=[roc_auc,conf_matrix,recall,accuracy]

    return test_scores

In [ ]:
#Prediction size may be smaller than y_test if 4 hours weren't enough to predict entire test set
pred_df=pd.DataFrame.from_dict(predictions)

if len(y_test)!= len(pred_df):
    y_test_cropped=y_test[:len(pred_df)]
    test_scores=GetFinalScores(pred_df,y_test_cropped)
else:
    test_scores=GetFinalScores(pred_df,y_test)

In [ ]:
test_scores

In [ ]:
test_ids = []
for v in test.StudyInstanceUID:
    if v not in test_ids:
        test_ids.append(v)

test_preds =pd.DataFrame(predictions)
test_preds.insert(0, "StudyInstanceUID", test.StudyInstanceUID.values)
test_preds

# Aggregate image-level predictions to exam-level

Now, we have image-level predictions.
Each Series contains several images, and each Study/Exam contains several Series of Images.
We need the Exam level labels.

For that, we need to aggregate the image-level labels.

- First task, define the label *negative_exam_for_pe*, which defines if the total exam is positive or negative for PE. To do this:
    -  Check for any image level presence of PE of high confidence (threshold of 0.5). If any image has pe_presence>0.5, its positive, otherwise its negative.
    -  If positive, set the study/exam level label of indeterminate to half the minimum probability of indeterminate image-level label and the *negative_exam_for_pe to 0*.
    -  If negative, the study/exam level label can be indeterminate or negative. If any image level label of indeterminate is above 0.5 (confidently indeterminate), set the study/exam level label of indeterminate to the max probability of indeterminate image-level label and the *negative_exam_pe* to 1. Otherwise the exam is negative, the indeterminate study/exam level label is set to half the minimum and the *negative_exam_pe* is set to 1.
    
    
- Second task, define the exam level label of severity: *chronic or acute_chronic_pe* and if none of these labels are confident, set to *acute*.
    -  Check for any image level labels of acute_chronic and chronic of high confidence (threshold of 0.5).
    -  If chronic, to agggregate the image level labels, set the exam-level label of chronic to 0.5+mean(image-level chronic label)/2 and the examl-level label of acute_chronic to mean(image-level acute_chronic label)/2. Similar is done if any image level label positive for acute_chronic with confidence (above 0.5)
    - If none, we set the exam level label to acute, by setting the exam level label of chronic and acute_chronic to mean(image-level)/2

- Third task, define the exam level label of position
    -  Same process as before but with the labels, right, left and center.

In [ ]:
from scipy.special import softmax

label_agg = {key:[] for key in 
             ['id', 'negative_exam_for_pe',
              'leftsided_pe', 'chronic_pe',
              'rightsided_pe', 'acute_and_chronic_pe',
              'central_pe', 'indeterminate']
            }

# label_agg = {key:[] for key in 
#              ['id', 'negative_exam_for_pe', 'rv_lv_ratio_gte_1',
#               'rv_lv_ratio_lt_1', 'leftsided_pe', 'chronic_pe',
#               'rightsided_pe', 'acute_and_chronic_pe',
#               'central_pe', 'indeterminate']
#             }

for uid in test_ids:
    #to the label aggregation dataframe, we also add the studyinstance to which we refer in the field id
    temp = test_preds.loc[test_preds.StudyInstanceUID == uid]
    label_agg['id'].append(uid)
    
    n = temp.shape[0]
    #Check for any image level presence of PE of high confidence
    positive = any(temp.pe_present_on_image >= 0.5) #50% threshhold
    
    #Only one from positive, negative and indeterminate should have value>0.5
    #per exam
    if positive: 
        #if positive, set the study/exam level label of indeterminate to half the minimum 
        #and the negative_exam_for_pe to 0 (n)
        label_agg['indeterminate'].append(temp.indeterminate.min()/2) 
        label_agg['negative_exam_for_pe'].append(0)
    else:
        #if negative i.e., no image in the study has more than 50% chance of presence of PE
        if any(temp.indeterminate >= 0.5):
            #it can be indeterminate, if any of the images has more than 50% chance of indeterminate
            label_agg['indeterminate'].append(temp.indeterminate.max())
            label_agg['negative_exam_for_pe'].append(1)
        else:
            #or it can be negative otherwise
            label_agg['indeterminate'].append(temp.indeterminate.min()/2)
            label_agg['negative_exam_for_pe'].append(1)
    
    #I decided that the total ratio should be equal to 1, so I used softmax
    #We modify the weights by multiplying the bigger by 2 and dividing the smaller by 2
#     a, b = temp[['rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1']].mean().values
#     if a > b:
#         a, b = a*2, b/2
#     elif a < b:
#         a, b = a/2, b*2
#     a, b = softmax([a, b])
#     if positive:
#         label_agg['rv_lv_ratio_gte_1'].append(a)
#         label_agg['rv_lv_ratio_lt_1'].append(b)
#     else:
#         label_agg['rv_lv_ratio_gte_1'].append(a/2)
#         label_agg['rv_lv_ratio_lt_1'].append(b/2)
    
    #Next is for Chronic (C), Acute-Chronic (AC) and Acute (A) PE
    #We need to see if we got a high confidence value from either C or AC
    #If there is, we add it to a 50% based score for high confidence
    #and half weight for low confidence score
    if any(temp['acute_and_chronic_pe'] > 0.5): #50% confidence level
        label_agg['acute_and_chronic_pe'].append(0.5 + temp['acute_and_chronic_pe'].mean()/2)
        label_agg['chronic_pe'].append(temp['chronic_pe'].mean()/2)
        
    elif any(temp['chronic_pe'] > 0.5):
        label_agg['acute_and_chronic_pe'].append(temp['acute_and_chronic_pe'].mean()/2)
        label_agg['chronic_pe'].append(0.5 + temp['chronic_pe'].mean()/2)
        
    else: #Else, we set both to half values, as we declare the A as the value
        label_agg['acute_and_chronic_pe'].append(temp['acute_and_chronic_pe'].mean()/2)
        label_agg['chronic_pe'].append(temp['chronic_pe'].mean()/2)
    
    #for right, left, central, we use the same metric above
    for key in ['leftsided_pe', 'rightsided_pe', 'central_pe']:
        if positive:
            label_agg[key].append(0.5 + temp[key].mean()/2)
        else:
            label_agg[key].append(temp[key].mean()/2)

# Calculate the exam-level test scores

To do so, we assume that the true exam-level labels are the same as the image-level labels for the test set. If we take a look at the data, if one image of the exam has acute_chronic_pe=1, all the others have it too.

In [ ]:
label_agg=pd.DataFrame(label_agg)

y_test_agg=pd.DataFrame(y_test)
y_test_agg.insert(0,'StudyInstanceUID',test.StudyInstanceUID.values)
y_test_agg=y_test_agg.drop_duplicates(subset="StudyInstanceUID")
y_test_agg=y_test_agg.drop(columns=['pe_present_on_image','qa_motion',
                                    'qa_contrast', 'flow_artifact', 'rv_lv_ratio_gte_1', 
                                    'rv_lv_ratio_lt_1','true_filling_defect_not_pe'])
# for uid in y_test.StudyInstanceUID:
#     if uid not in y_test_agg:
#         y_test_agg.append(y_test.loc("S"))

test_scores_exam=GetFinalScores(label_agg,y_test_agg,True)


In [ ]:
test_scores_exam